In [13]:
import json
from pathlib import Path

CURATED_PATH = Path("/Users/james/Documents/obi/Data/Simulations/BBP-curated/assemblies")
RAW_CONFIGS_PATH = Path("/Users/james/Documents/obi/Data/Simulations/BBP-raw/assemblies/")
RAW_INPUT_OUTPUT_PATH = RAW_CONFIGS_PATH / "d21efd45-7740-4268-b06f-e1de35f2b6cf"

sim_length = 127500
ou_start_time = 250
ca_conc = 1.05

In [22]:
def add_relative_ou_stimulus(config, name, node_set, mean, sd, delay, duration):

    config['inputs'][name] = {
            "input_type": "conductance",
            "module": "relative_ornstein_uhlenbeck",
            "delay": delay,
            "duration": duration,
            "reversal": 0,
            "tau": 2.7,
            "mean_percent": mean,
            "sd_percent": sd,
            "node_set": node_set
        }
    
def add_spike_replay(config, name, sim_length, spike_file, source):
    config["inputs"]["Stimulus spikeReplay " + name] = {
            "module": "synapse_replay",
            "input_type": "spikes",
            "delay": 0,
            "duration": sim_length,
            "spike_file": spike_file,
            "source": source,
            "node_set": "hex_O1"
        }
    
def add_soma_reports(config, sim_length):
  config["reports"] = {
      "Report soma": {
        "cells": "hex_O1",
        "sections": "soma",
        "type": "compartment",
        "compartments": "center",
        "variable_name": "v",
        "unit": "mV",
        "dt": 1.0,
        "start_time": 0,
        "end_time": sim_length
      }
    }
  
def add_connection_overrides(config):
    config["connection_overrides"] = [
            {
                "name": "no_vpm_proj",
                "source": "proj_Thalamocortical_VPM_Source",
                "target": "hex_O1",
                "spontMinis": 0.0,
                "weight": 1.0
            },
            {
                "name": "no_pom_proj",
                "source": "proj_Thalamocortical_POM_Source",
                "target": "hex_O1",
                "spontMinis": 0.0,
                "weight": 1.0
            },
            {
                "name": "init",
                "source": "hex_O1",
                "target": "hex_O1",
                "weight": 1.0
            },
            {
                "name": "disconnect",
                "source": "hex_O1",
                "target": "hex_O1",
                "delay": 0.025,
                "weight": 0.0
            },
            {
                "name": "reconnect",
                "source": "hex_O1",
                "target": "hex_O1",
                "delay": 1000,
                "weight": 1.0
            }
        ]
    
def add_conditions(config, ca_conc):

    config["conditions"] = {
        "extracellular_calcium": ca_conc,
        "v_init": -80.0,
        "mechanisms": {
            "ProbAMPANMDA_EMS": {
                "init_depleted": True,
                "minis_single_vesicle": True
            },
            "ProbGABAAB_EMS": {
                "init_depleted": True,
                "minis_single_vesicle": True
            }
        }
    }

import h5py
import json
import numpy as np
from pathlib import Path
from time import perf_counter

_ID_MAPPING_CACHE = None
_ID_MAPPING_FILE = Path("/Users/james/Documents/obi/Data/Circuits/nbS1-O1/id_mapping.json")

def _load_id_mapping():
    global _ID_MAPPING_CACHE

    if _ID_MAPPING_CACHE is not None:
        return _ID_MAPPING_CACHE

    if not _ID_MAPPING_FILE.exists():
        print(f"Mapping file not found: {_ID_MAPPING_FILE}")
        _ID_MAPPING_CACHE = {}
        return _ID_MAPPING_CACHE

    t0 = perf_counter()
    with _ID_MAPPING_FILE.open("r", encoding="utf-8") as f:
        _ID_MAPPING_CACHE = json.load(f)

    # print(mapping_dict['S1nonbarrel_neurons']["old_id"][1:10])
    # _ID_MAPPING_CACHE = {int(k): int(v) for k, v in mapping_dict.items()}
    # print(
    #     f"Loaded {_ID_MAPPING_FILE.name} with {len(_ID_MAPPING_CACHE):,} ids "
    #     f"in {perf_counter() - t0:.2f}s"
    # )
    return _ID_MAPPING_CACHE

def adjust_spike_file(spike_path, output_spike_path, population, map_spikes_to_new_node_ids):

    print(f"Adjusting spike file: {spike_path}")

    with h5py.File(spike_path, "r") as fin:
        node_ids = fin[f"spikes/{population}/node_ids"][:]       # (N,)
        timestamps = fin[f"spikes/{population}/timestamps"][:]   # (N,)

    # new_node_ids = node_ids
    if map_spikes_to_new_node_ids:
        mapping_dict = _load_id_mapping()
        if mapping_dict:
            pop_old_ids = mapping_dict.get(population, {}).get("old_id", {})
            pop_old_ids_dict = {old_id: new_id for new_id, old_id in enumerate(pop_old_ids)}
            new_node_ids = np.asarray([pop_old_ids_dict[node_id] for node_id in node_ids])
            # print(new_node_ids[:10])
            changed = int(np.count_nonzero(new_node_ids != node_ids))
            print(f"Remapped {changed:,}/{len(node_ids):,} spike node ids")

        reporting_dir = output_spike_path.parent
        reporting_dir.mkdir(exist_ok=True)
        with h5py.File(output_spike_path, "w") as f:

            g_spikes = f.create_group("spikes")
            g_pom = g_spikes.create_group("S1nonbarrel_neurons")

            g_pom.create_dataset("node_ids", data=new_node_ids)
            g_pom.create_dataset("timestamps", data=timestamps)

In [23]:
indices_and_seeds = [(0, 1), (1, 19), (2, 31), (3, 39), (4, 47), (5, 54), (6, 63), (7, 78), (8, 80), (9, 99)]

for index, random_seed in indices_and_seeds[0:5]:

    config = {
        "version": 1,
        "run": {
            "dt": 0.025,
            "random_seed": random_seed,
            "tstop": sim_length
        },
        "spike_location": "AIS",
        "network": "../circuit_config.json",
        "node_set": "hex_O1",
        "node_set_file": "node_sets.json",
        "target_simulator": "CORENEURON",
        "output": {
            "output_dir": "./reporting",
            "spikes_file": "spikes.h5"
        },
        "inputs": {},
        "reports": {}
    }

    add_relative_ou_stimulus(config, "Stimulus gExc_L1", "Layer1", 2.993, 1.197, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L23E", "Layer23Excitatory", 18.183, 7.273, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L23I", "Layer23Inhibitory", 2.587, 1.035, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L4E", "Layer4Excitatory", 8.844, 3.538, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L4I", "Layer4Inhibitory", 3.13, 1.252, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L5E", "Layer5Excitatory", 16.394, 6.558, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L5I", "Layer5Inhibitory", 4.761, 1.904, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L6E", "Layer6Excitatory", 2.562, 1.025, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L6I", "Layer6Inhibitory", 3.026, 1.21, ou_start_time, sim_length)

    add_spike_replay(config, "VPM", sim_length, "input_VPM.h5", "proj_Thalamocortical_VPM_Source")
    add_spike_replay(config, "POm", sim_length, "input_POM.h5", "proj_Thalamocortical_POM_Source")
    # add_soma_reports(config, sim_length)
    add_connection_overrides(config)
    add_conditions(config, ca_conc)

    curated_sim_dir = CURATED_PATH / str(index)
    curated_sim_dir.mkdir(exist_ok=True, parents=True)
    config_path = curated_sim_dir / Path(f"simulation_config.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=4)

    adjust_spike_file(CURATED_PATH / str(index) / "spikes.h5", CURATED_PATH / str(index) / "reporting" / "spikes.h5", "S1nonbarrel_neurons", True)
    

Adjusting spike file: /Users/james/Documents/obi/Data/Simulations/BBP-curated/assemblies/0/spikes.h5
Remapped 14,436,988/14,436,988 spike node ids
Adjusting spike file: /Users/james/Documents/obi/Data/Simulations/BBP-curated/assemblies/1/spikes.h5
Remapped 14,441,051/14,441,051 spike node ids
Adjusting spike file: /Users/james/Documents/obi/Data/Simulations/BBP-curated/assemblies/2/spikes.h5
Remapped 14,417,765/14,417,765 spike node ids
Adjusting spike file: /Users/james/Documents/obi/Data/Simulations/BBP-curated/assemblies/3/spikes.h5
Remapped 14,436,096/14,436,096 spike node ids
Adjusting spike file: /Users/james/Documents/obi/Data/Simulations/BBP-curated/assemblies/4/spikes.h5
Remapped 14,469,209/14,469,209 spike node ids
